# CLM5-BGC OAAT parameter ensemble
- generates paramfiles and namelist_mods for the OAAT segment of the CLM5PPE
- leverages ppe-tools to write out the files
- written with community-derived ensemble ranges:
    - https://docs.google.com/spreadsheets/d/e/2PACX-1vQs413GtLXtHVDCqEPgAwn4BbDjoWmV7uFqOAWH4mgpxXoVfN6ijnJdhyRgLkV-n2eU-sSQush4CzYU
- Katie Dagon (kdagon@ucar.edu) and Daniel Kennedy (djk2120@ucar.edu)

In [12]:
import numpy as np
from ppe_tools import Ensemble, Member, ParamInfo
import xarray as xr
import os
import pandas as pd

### read in the google spreadsheet and and organize it into a dataframe

In [13]:
#data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQs413GtLXtHVDCqEPgAwn4BbDjoWmV7uFqOAWH4mgpxXoVfN6ijnJdhyRgLkV-n2eU-sSQush4CzYU/pub?output=csv'
#cmd = 'curl -L '+data_url+' > params.csv' # need to add -L option to force redirects
#os.system(cmd)

In [14]:
csvfile = 'params.csv'
data = pd.read_csv(csvfile,header=0,skiprows=[1]) # modify read_csv to account for header spanning 2 rows
included = data['final'] == 1 # final flag
params_full = data.loc[included,['name','location','min','max','flag','pft_mins','pft_maxs']]
params = params_full.reset_index(drop=True) # reset indexing and get rid of excel row number

### create a dictionary of all the oaat perturbation
 - parameter matched with its min and max values

In [4]:
oaats = {}
names = params['name']
flags = params['flag']

for name,flag in zip(names,flags):
    if pd.notnull(flag):
        if flag=='KCN':
            flag=False
        else:
            ff = flag
            flag=True
    else: flag=False
    if not flag:
        ix = params['name']==name
        minval = params['min'][ix].values[0]
        maxval = params['max'][ix].values[0]
        pftmin = params['pft_mins'][ix].values[0]
        pftmax = params['pft_maxs'][ix].values[0]
        thisloc = params['location'][ix].values[0]
        
        needs_pft = (minval=='pft')
        if needs_pft:
            thismin = np.fromstring(pftmin, dtype='float', sep=',')
        elif 'percent' in minval:
            thismin = minval
        else:
            thismin = np.array(float(minval))

        needs_pft = (maxval=='pft')
        if needs_pft:
            thismax = np.fromstring(pftmax, dtype='float', sep=',')
        elif 'percent' in maxval:
            thismax = maxval
        else:
            thismax = np.array(float(maxval))

        oaats[name]={'min':thismin,'max':thismax,'loc':thisloc}

### create the ensemble object and populate it with the oaats

In [5]:
basefile = '/glade/p/cgd/tss/people/oleson/modify_param/ctsm51_params.c210217_kwo.c210222.nc'
pdir = '/glade/u/home/djk2120/ppetest/paramfiles/'
ndir = '/glade/u/home/djk2120/ppetest/namelist_mods/'
x = Ensemble(basefile,pdir,ndir)
prefix = 'OAAT'
nextnum = 1
x.add_oaats(oaats,prefix,nextnum,skipBFB=True)

a_coef-max looks BFB.... skipping
a_exp-min looks BFB.... skipping
lai_dl-min looks BFB.... skipping
interception_fraction-max looks BFB.... skipping
aq_sp_yield_min-max looks BFB.... skipping
n_baseflow-min looks BFB.... skipping
accum_factor-min looks BFB.... skipping
wind_snowcompact_fact-min looks BFB.... skipping
theta_ip-min looks BFB.... skipping
fnr-max looks BFB.... skipping
cn_s1_bgc-min looks BFB.... skipping
decomp_depth_efolding-max looks BFB.... skipping


In [6]:
x.nmemb

378

### add in the special cases

In [7]:
#these params move in unison
flags = params['flag']
ix = pd.notnull(params['flag'])
uflags = pd.unique(flags[ix])
sgns = {'min':'-','max':''}
for uflag in uflags:
    names = params['name'][flags==uflag]
    for minmax in ['min','max']:
        mf = {}
        for name in names:
            ix = params['name']==name

            thisval = params[minmax][ix].values[0]
            pftval  = params['pft_'+minmax+'s'][ix].values[0]
            thisloc = params['location'][ix].values[0]

            needs_pft = (thisval=='pft')
            if needs_pft:
                val = np.fromstring(pftval, dtype='float', sep=',')
            elif 'percent' in thisval:
                val = sgns[minmax]+thisval
            else:
                val = np.array(float(thisval))

            mf[name] = {'value':val,'loc':thisloc,'minmax':minmax,'flag':uflag}

        x.add_mf(mf,'OAAT')


In [8]:
x.nmemb

388

In [9]:
csvfile = '/glade/u/home/djk2120/ppetest/OAAT_apr2020.csv'
default_key='OAAT0000'
x.write(default_key,csvfile)

### spot check a few variables?

In [20]:
#should only have a min
thisvar='a_coef'
for member in x.members:
    if thisvar in member.paramdict:
        f = pdir+member.name+'.nc'
        params = [*member.paramdict]
        if len(params)==1:
            param=params[0]
        else:
            param=member.flag
        print(member.name,param,member.minmax)
        p = xr.open_dataset(f)
        for var in member.paramdict:
            print(p[var])

OAAT0029 a_coef min
<xarray.DataArray 'a_coef' ()>
array(0.1)
Attributes:
    long_name:  Drag coeff. under less dense canopy
    units:      unitless


In [16]:
#handling namelist?
thisvar='interception_fraction'
for key in ['OAAT'+str(i).zfill(4) for i in range(389)]:
    nlfile = ndir+key+'.txt'
    with open(nlfile,"r") as file:
        ff = file.read()
    if thisvar in ff:
        print(ff)

! user_nl_clm namelist options written by generate_params:
interception_fraction=0.5



In [17]:
#handling PFT?
thisvar='dleaf'
for member in x.members:
    if thisvar in member.paramdict:
        f = pdir+member.name+'.nc'
        params = [*member.paramdict]
        if len(params)==1:
            param=params[0]
        else:
            param=member.flag
        print(member.name,param,member.minmax)
        p = xr.open_dataset(f)
        for var in member.paramdict:
            print(p[var])

OAAT0021 dleaf min
<xarray.DataArray 'dleaf' (pft: 79)>
array([0.      , 0.000216, 0.000216, 0.00072 , 0.0081  , 0.0081  , 0.0081  ,
       0.0081  , 0.0081  , 0.0081  , 0.000405, 0.000162, 0.000144, 0.000144,
       0.000144, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162,
       0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162,
       0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162,
       0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162,
       0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162,
       0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162,
       0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162,
       0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162,
       0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162, 0.000162,
       0.000162, 0.000162])
Coordinates:
    pftname  (pft) |S40 ...
Dimensions without coordinat

In [18]:
#handling PFTxsegment?
thisvar='ck'
for member in x.members:
    if thisvar in member.paramdict:
        f = pdir+member.name+'.nc'
        params = [*member.paramdict]
        if len(params)==1:
            param=params[0]
        else:
            param=member.flag
        print(member.name,param,member.minmax)
        p = xr.open_dataset(f)
        for var in member.paramdict:
            print(p[var])

OAAT0165 ck min
<xarray.DataArray 'ck' (segment: 4, pft: 79)>
array([[3., 3., 3., ..., 3., 3., 3.],
       [3., 3., 3., ..., 3., 3., 3.],
       [3., 3., 3., ..., 3., 3., 3.],
       [3., 3., 3., ..., 3., 3., 3.]])
Coordinates:
    pftname  (pft) |S40 ...
  * segment  (segment) |S40 b'sunlit                                  ' ... b...
Dimensions without coordinates: pft
Attributes:
    units:      unitless
    long_name:  weibull curve shape parameter
OAAT0166 ck max
<xarray.DataArray 'ck' (segment: 4, pft: 79)>
array([[5.15, 5.15, 5.15, ..., 5.15, 5.15, 5.15],
       [5.15, 5.15, 5.15, ..., 5.15, 5.15, 5.15],
       [5.15, 5.15, 5.15, ..., 5.15, 5.15, 5.15],
       [5.15, 5.15, 5.15, ..., 5.15, 5.15, 5.15]])
Coordinates:
    pftname  (pft) |S40 ...
  * segment  (segment) |S40 b'sunlit                                  ' ... b...
Dimensions without coordinates: pft
Attributes:
    units:      unitless
    long_name:  weibull curve shape parameter


In [22]:
#should have kc_nonmyc min/max AND KCN min/max
thisvar='kc_nonmyc'
for member in x.members:
    if thisvar in member.paramdict:
        f = pdir+member.name+'.nc'
        params = [*member.paramdict]
        if len(params)==1:
            param=params[0]
        else:
            param=member.flag
        print(member.name,param,member.minmax)
        p = xr.open_dataset(f)
        for var in member.paramdict:
            print(p[var])

OAAT0193 kc_nonmyc min
<xarray.DataArray 'kc_nonmyc' (pft: 79)>
array([0.    , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.0072, 0.072 , 0.072 ,
       0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.72  , 0.072 , 0.072 , 0.72  ,
       0.72  , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 ,
       0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 ,
       0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 ,
       0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 ,
       0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 , 0.072 ,
       0.072 , 0.072 , 0.072 , 0.072 , 0.72  , 0.72  , 0.072 , 0.072 , 0.072 ,
       0.072 , 0.72  , 0.72  , 0.72  , 0.72  , 0.072 , 0.072 ])
Coordinates:
    pftname  (pft) |S40 ...
Dimensions without coordinates: pft
Attributes:
    long_name:  Constant relating root C to non-mycorrhizal root active uptak...
    units:      gC/m3
OAAT0194 kc_nonmyc max
<xarray.DataArray 'kc_non